<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/184-Mini-Project-H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv langchain langchain_openai langchain_community langchain_chroma langchain_text_splitters

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

1. Sample texts:

texts 리스트에는 샘플 텍스트들이 포함되어 있습니다. 각 텍스트는 광대버섯(Amanita phalloides)에 대한 정보를 제공합니다.
Custom text loader function:

2. Custom text loader function:

load_texts 함수는 주어진 텍스트 리스트를 받아서 Document 객체의 리스트로 변환합니다. Document 객체는 페이지 콘텐츠(텍스트 내용)와 메타데이터를 포함합니다.

3.Load the documents:

load_texts 함수를 사용하여 raw_documents 변수에 문서를 로드합니다.

4. Initialize the text splitter and embeddings:

CharacterTextSplitter 객체와 OpenAIEmbeddings 객체를 초기화합니다. CharacterTextSplitter는 텍스트를 일정 크기의 조각으로 분할하고, OpenAIEmbeddings는 텍스트 조각을 임베딩 벡터로 변환하는 역할을 합니다.

5. Split the documents into chunks:

각 문서를 100자 크기의 텍스트 조각으로 분할합니다. chunk_overlap이 0으로 설정되어 있으므로, 조각 사이에 중복이 없습니다.

6. Embed each chunk:

각 텍스트 조각을 임베딩합니다. 임베딩은 텍스트 조각을 벡터로 변환하여 기계 학습 모델이 이해할 수 있도록 합니다.

7. Initialize the vector store with embedded chunks:

Chroma 벡터 스토어를 초기화하고, 임베딩된 텍스트 조각들을 저장합니다. 이 벡터 스토어는 나중에 유사한 텍스트 검색을 수행하는 데 사용됩니다.

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document  # Import the Document class

# Sample texts
texts = [
    "광대버섯(Amanita phalloides)은 크고 인상적인 후성(위) 자실체(담자과체)를 가지고 있습니다.",
    "큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.",
    "A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.",
]

# Custom text loader function
def load_texts(text_list):
    documents = []
    for text in text_list:
        documents.append(Document(page_content=text, metadata={}))  # Create Document objects
    return documents

# Load the documents
raw_documents = load_texts(texts)

# Initialize the text splitter and embeddings
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
embeddings = OpenAIEmbeddings()

# Split the documents into chunks
chunks = []
for doc in raw_documents:
    chunks.extend(text_splitter.split_text(doc.page_content))

# Embed each chunk
embedded_chunks = embeddings.embed_documents([chunk for chunk in chunks])  # Pass list of strings

# Initialize the vector store with embedded chunks
vector_store = Chroma.from_documents(
    [Document(page_content=chunk, metadata={}) for chunk in chunks],
    embeddings
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# def dynamic_search(question):
#     try:
#         k = int(input("원하는 검색 결과의 수를 입력하세요 (1-3): "))
#         if k < 1 or k > 3:
#             raise ValueError("검색 결과의 수는 1에서 3 사이여야 합니다.")
#     except ValueError as e:
#         print(f"잘못된 입력입니다: {e}")
#         return []

#     # Ensure vector_store is properly initialized and accessible
#     return vector_store.similarity_search_with_score(query=question, k=k)

### 유사도 기반 검색 수행:

retriever.get_relevant_documents(question)를 호출하여 질문에 대한 관련 문서를 검색합니다. 이 검색은 유사도 점수 임계값을 기준으로 필터링된 결과를 반환합니다.

In [13]:
# Initialize the retriever with a similarity score threshold
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9} # score_threshold": 0.5, 0.7 0.9로 조절하여 정답 갯수를 확인
)

### score_threshold": 0.9를 조절하여 정답 갯수를 확인

In [14]:
def dynamic_search_with_threshold(question):
    """

    Args:
      question:

    Returns:

    """
    try:
        k = int(input("원하는 검색 결과의 수를 입력하세요 (1-3): "))
        if k < 1 or k > 3:
            raise ValueError("검색 결과의 수는 1에서 3 사이여야 합니다.")
    except ValueError as e:
        print(f"잘못된 입력입니다: {e}")
        return []

    # Perform the similarity search with the retriever
    results = retriever.get_relevant_documents(question)

    # Limit the results to the top k items
    return results[:k]

# Example usage
question = "광대버섯은 무엇입니까?"
results = dynamic_search_with_threshold(question)
for result in results:
    print(result)


원하는 검색 결과의 수를 입력하세요 (1-3): 3


/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(
